In [1]:
from subprocess import Popen, PIPE

In [2]:
import pandas

In [3]:
import re
reg_exec_time = re.compile(rb'Execution time: (\d*\.?\d*)us\n')

In [4]:
from collections import OrderedDict

In [5]:
# note: all byte ops are in hex

In [6]:
class Op:
    OP_0 = '00'
    OP_FALSE = OP_0
    OP_PUSHDATA1 = '4c'
    OP_PUSHDATA2 = '4d'
    OP_PUSHDATA4 = '4e'
    OP_1NEGATE = '4f'
    OP_RESERVED = '50'
    OP_1 = '51'
    OP_TRUE = OP_1
    OP_2 = '52'
    OP_3 = '53'
    OP_4 = '54'
    OP_5 = '55'
    OP_6 = '56'
    OP_7 = '57'
    OP_8 = '58'
    OP_9 = '59'
    OP_10 = '5a'
    OP_11 = '5b'
    OP_12 = '5c'
    OP_13 = '5d'
    OP_14 = '5e'
    OP_15 = '5f'
    OP_16 = '60'

    # control
    OP_NOP = '61'
    OP_VER = '62'
    OP_IF = '63'
    OP_NOTIF = '64'
    OP_VERIF = '65'
    OP_VERNOTIF = '66'
    OP_ELSE = '67'
    OP_ENDIF = '68'
    OP_VERIFY = '69'
    OP_RETURN = '6a'

    # stack ops
    OP_TOALTSTACK = '6b'
    OP_FROMALTSTACK = '6c'
    OP_2DROP = '6d'
    OP_2DUP = '6e'
    OP_3DUP = '6f'
    OP_2OVER = '70'
    OP_2ROT = '71'
    OP_2SWAP = '72'
    OP_IFDUP = '73'
    OP_DEPTH = '74'
    OP_DROP = '75'
    OP_DUP = '76'
    OP_NIP = '77'
    OP_OVER = '78'
    OP_PICK = '79'
    OP_ROLL = '7a'
    OP_ROT = '7b'
    OP_SWAP = '7c'
    OP_TUCK = '7d'

    # splice ops
    OP_CAT = '7e'
    OP_SPLIT = '7f'   # after monolith upgrade (May 2018)
    OP_NUM2BIN = '80' # after monolith upgrade (May 2018)
    OP_BIN2NUM = '81' # after monolith upgrade (May 2018)
    OP_SIZE = '82'

    # bit logic
    OP_INVERT = '83'
    OP_AND = '84'
    OP_OR = '85'
    OP_XOR = '86'
    OP_EQUAL = '87'
    OP_EQUALVERIFY = '88'
    OP_RESERVED1 = '89'
    OP_RESERVED2 = '8a'

    # numeric
    OP_1ADD = '8b'
    OP_1SUB = '8c'
    OP_2MUL = '8d'
    OP_2DIV = '8e'
    OP_NEGATE = '8f'
    OP_ABS = '90'
    OP_NOT = '91'
    OP_0NOTEQUAL = '92'

    OP_ADD = '93'
    OP_SUB = '94'
    OP_MUL = '95'
    OP_DIV = '96'
    OP_MOD = '97'
    OP_LSHIFT = '98'
    OP_RSHIFT = '99'

    OP_BOOLAND = '9a'
    OP_BOOLOR = '9b'
    OP_NUMEQUAL = '9c'
    OP_NUMEQUALVERIFY = '9d'
    OP_NUMNOTEQUAL = '9e'
    OP_LESSTHAN = '9f'
    OP_GREATERTHAN = 'a0'
    OP_LESSTHANOREQUAL = 'a1'
    OP_GREATERTHANOREQUAL = 'a2'
    OP_MIN = 'a3'
    OP_MAX = 'a4'

    OP_WITHIN = 'a5'

    # crypto
    OP_RIPEMD160 = 'a6'
    OP_SHA1 = 'a7'
    OP_SHA256 = 'a8'
    OP_HASH160 = 'a9'
    OP_HASH256 = 'aa'
    OP_CODESEPARATOR = 'ab'
    OP_CHECKSIG = 'ac'
    OP_CHECKSIGVERIFY = 'ad'
    OP_CHECKMULTISIG = 'ae'
    OP_CHECKMULTISIGVERIFY = 'af'

    # expansion
    OP_NOP1 = 'b0'
    OP_CHECKLOCKTIMEVERIFY = 'b1'
    OP_NOP2 = OP_CHECKLOCKTIMEVERIFY
    OP_CHECKSEQUENCEVERIFY = 'b2'
    OP_NOP3 = OP_CHECKSEQUENCEVERIFY
    OP_NOP4 = 'b3'
    OP_NOP5 = 'b4'
    OP_NOP6 = 'b5'
    OP_NOP7 = 'b6'
    OP_NOP8 = 'b7'
    OP_NOP9 = 'b8'
    OP_NOP10 = 'b9'

    # More crypto
    OP_CHECKDATASIG = 'ba'
    OP_CHECKDATASIGVERIFY = 'bb'

    # multi-byte opcodes
    OP_PREFIX_BEGIN = 'f0'
    OP_PREFIX_END = 'f7'

In [7]:
def push_int_hex(n):
    assert n >= 0
    op_name = f'OP_{n}'
    if hasattr(Op, op_name):
        return getattr(Op, op_name)
    b = n.to_bytes(16, 'little')
    b = b.rstrip(b'\0')
    return '%02x' % len(b) + b.hex()

In [8]:
def varint_hex(n):
    if n < 0xfd:
        return '%02x' % n
    elif n <= 0xffff:
        return 'fd' + n.to_bytes(length=2, byteorder='little').hex()
    elif n <= 0xffff_ffff:
        return 'fe' + n.to_bytes(length=4, byteorder='little').hex()
    else:
        return 'ff' + n.to_bytes(length=8, byteorder='little').hex()

In [9]:
from dataclasses import dataclass
@dataclass
class LimitParams:
    MAX_SCRIPT_ELEMENT_SIZE: int = None
    MAX_OPS_PER_SCRIPT: int = None
    MAX_PUBKEYS_PER_MULTISIG: int = None
    MAX_SCRIPT_SIZE: int = None
    MAX_STACK_SIZE: int = None
    ENABLE_OP_MUL: bool = False
    ENABLE_INT128: bool = False
    def param_list(self):
        params = []
        if self.MAX_SCRIPT_ELEMENT_SIZE is not None:
            params.append(f'-MAX_SCRIPT_ELEMENT_SIZE={self.MAX_SCRIPT_ELEMENT_SIZE}')
        if self.MAX_OPS_PER_SCRIPT is not None:
            params.append(f'-MAX_OPS_PER_SCRIPT={self.MAX_OPS_PER_SCRIPT}')
        if self.MAX_PUBKEYS_PER_MULTISIG is not None:
            params.append(f'-MAX_PUBKEYS_PER_MULTISIG={self.MAX_PUBKEYS_PER_MULTISIG}')
        if self.MAX_SCRIPT_SIZE is not None:
            params.append(f'-MAX_SCRIPT_SIZE={self.MAX_SCRIPT_SIZE}')
        if self.MAX_STACK_SIZE is not None:
            params.append(f'-MAX_STACK_SIZE={self.MAX_STACK_SIZE}')
        if self.ENABLE_OP_MUL:
            params.append('-ENABLE_OP_MUL')
        if self.ENABLE_INT128:
            params.append('-ENABLE_INT128')
        return params
    def max_script_element_size(self):
        return self.MAX_SCRIPT_ELEMENT_SIZE or 520
    def max_ops_per_script(self):
        return self.MAX_OPS_PER_SCRIPT or 201
    def max_pubkeys_per_multisig(self):
        return self.MAX_PUBKEYS_PER_MULTISIG or 20
    def max_script_size(self):
        return self.MAX_SCRIPT_SIZE or 10_000
    def max_stack_size(self):
        return self.MAX_STACK_SIZE or 1000

In [10]:
def bench_script(amount, script_sig, script_pub_key=None, limit_params=None, locktime=None, sequence=None):
    version = '01000000'
    if locktime is None:
        locktime = '00000000'
    else:
        locktime = locktime.to_bytes(4, 'little').hex()
    if sequence is None:
        sequence = 'ffffffff'
    else:
        sequence = sequence.to_bytes(4, 'little').hex()
    limit_params = limit_params or LimitParams()
    
    tx = (
        version +
        '01' + # one input
        '77'*32 + '00'*4 + # 77..77:0, random outpoint
        varint_hex(len(script_sig) // 2) + # len of script
        script_sig +
        sequence +
        '00' + # no outputs
        locktime
    )
    
    params = (
        ['./build/src/bitcoin-tx'] +
        limit_params.param_list() +
        [tx] +
        ['benchin=0:' + '{:.8f}'.format(amount) + (':' + script_pub_key if script_pub_key is not None else '')]
    )
    
    process = Popen(
        params,
        stdout=PIPE,
        stderr=PIPE,
    )
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code:
        print(output.decode())
        raise ValueError(f'{err} for params {params}')
    else:
        m = reg_exec_time.match(output)
        if m is None:
            raise ValueError(f'unexpected output: {output.decode()} for params {params}')
        return float(m.group(1))

In [11]:
def bench_tx(tx, amount, benchin, limit_params=None):
    limit_params = limit_params or LimitParams()
    params = (
        ['./build/src/bitcoin-tx'] +
        limit_params.param_list() +
        [tx] +
        ['benchin=0:' + '{:.8f}'.format(amount)]
    )
    
    process = Popen(
        params,
        stdout=PIPE,
        stderr=PIPE,
    )
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code:
        print(output.decode())
        raise ValueError(f'{err} for params {params}')
    else:
        return output

In [12]:
def make_big_if_scripts(limit_params):
    n_ifs = limit_params.max_ops_per_script() // 2
    n_zero = limit_params.max_script_size() - 2 - (n_ifs * 2)
    script = [Op.OP_0]
    for _ in range(n_ifs):
        script += [Op.OP_IF]
    for _ in range(n_zero):
        script += [Op.OP_0]
    for _ in range(n_ifs):
        script += [Op.OP_ENDIF]
    return Op.OP_TRUE, ''.join(script)

In [13]:
def make_big_int128_func(mode: str):
    def make_big_int128_script(limit_params):
        x = 0x7fff_ffff_ffff_ffff_ffff_ffff_ffff_ffff
        y = 0x1234_5678_abcd_ef12
        z = x // y
        script = [
            push_int_hex(x),
            push_int_hex(y),
            push_int_hex(z),
        ]
        n_arith = (limit_params.max_ops_per_script() - 1) // 4
        for i in range(n_arith):
            script += [
                Op.OP_3DUP, 
                Op.OP_MUL,
                Op.OP_DIV, 
                Op.OP_MUL if mode == 'mul' else Op.OP_MUL,
            ]
        script += [
            Op.OP_2DROP,
        ]
        return '', ''.join(script)
    make_big_int128_script.__name__ = f'make_big_int128_script_{mode}'
    return make_big_int128_script

In [14]:
limit_params = LimitParams(ENABLE_OP_MUL=True, ENABLE_INT128=True)
script_sig, script_pub_key = make_big_int128_func('mul')(LimitParams(limit_params))
bench_script(1, script_sig, script_pub_key, limit_params)

164.21

In [15]:
tests = [
    (make_big_if_scripts,            LimitParams()),
    (make_big_int128_func('mul'),    LimitParams(ENABLE_OP_MUL=True, ENABLE_INT128=True)),
    (make_big_int128_func('div'),    LimitParams(ENABLE_OP_MUL=True, ENABLE_INT128=True)),
]

In [16]:
opcode_limits = [201, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000]

In [17]:
block_size = 32_000_000
sig_check_dt = 70
max_sig_check_dt = (block_size / 100) * sig_check_dt / 1_000_000

In [18]:
results = []
script_sizes = []
block_exec_times = []
for opcode_limit in opcode_limits:
    row = OrderedDict()
    row['opcode_limit'] = opcode_limit
    row_size = OrderedDict()
    row_size['opcode_limit'] = opcode_limit
    row_block = OrderedDict()
    row_block['opcode_limit'] = opcode_limit
    for f, limit_params in tests:
        limit_params.MAX_OPS_PER_SCRIPT = opcode_limit
        script_sig, script_pub_key = f(limit_params)
        dt = bench_script(1, script_sig, script_pub_key, limit_params)
        name = f.__name__.replace('make_', '')
        row[name] = dt
        size = len(script_pub_key) // 2
        row_size[name] = size
        row_block[name + ' p2sh'] = (block_size // (size + 41)) * dt / 1_000_000
        row_block[name + ''] = (block_size // 41) * dt / 1_000_000
    results.append(row)
    script_sizes.append(row_size)
    block_exec_times.append(row_block)
df = pandas.DataFrame(results)
df_size = pandas.DataFrame(script_sizes)
df_block_exec = pandas.DataFrame(block_exec_times)

In [19]:
df_size

,opcode_limit,big_if_scripts,big_int128_script_mul,big_int128_script_div
0,201,9999,237,237
1,300,9999,333,333
2,400,9999,433,433
3,500,9999,533,533
4,600,9999,633,633
5,700,9999,733,733
6,800,9999,833,833
7,900,9999,933,933
8,1000,9999,1033,1033
9,2000,9999,2033,2033


In [20]:
df_block_exec

,opcode_limit,big_if_scripts p2sh,big_if_scripts,big_int128_script_mul p2sh,big_int128_script_mul,big_int128_script_div p2sh,big_int128_script_div
0,201,0.674433,165.166659,17.441013,118.259390,17.765614,120.460364
1,300,0.689507,168.858362,18.643742,170.068117,18.714758,170.715922
2,400,0.741360,181.556886,20.635107,238.563656,21.532990,248.944134
3,500,0.760641,186.278832,20.210685,282.949952,20.105877,281.482637
4,600,0.699833,171.387140,21.602985,355.137195,22.639407,372.175226
5,700,0.808606,198.025162,20.985293,396.167396,20.979919,396.065933
6,800,0.805674,197.307114,23.710945,505.451186,21.550778,459.402453
7,900,0.892806,218.645628,20.971365,498.200462,20.498925,486.977059
8,1000,0.842707,206.376373,21.135083,553.638453,21.077281,552.124309
9,2000,0.990488,242.567555,22.767958,1151.733446,21.859961,1105.801786


In [21]:
df

,opcode_limit,big_if_scripts,big_int128_script_mul,big_int128_script_div
0,201,211.62,151.52,154.34
1,300,216.35,217.90,218.73
2,400,232.62,305.66,318.96
3,500,238.67,362.53,360.65
4,600,219.59,455.02,476.85
5,700,253.72,507.59,507.46
6,800,252.80,647.61,588.61
7,900,280.14,638.32,623.94
8,1000,264.42,709.35,707.41
9,2000,310.79,1475.66,1416.81
